In [ ]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
cohort_id = 'ML-2_Kati_colu_2014_Q3'
cohorts_analysis = "20230223"
contigs = ['3L']
sample_sets = "3.0"
min_cohort_size = 20
max_cohort_size = 50
h12_calibration_contig = '3L'
use_gcs_cache = False
dask_scheduler = "threads"

In [ ]:
from IPython.display import Markdown, HTML
import malariagen_data
import numpy as np
import pandas as pd
from pyprojroot import here
import yaml
import dask
dask.config.set(scheduler=dask_scheduler);
from textwrap import dedent

import geopandas as gpd
import bokeh.layouts as bklay
import bokeh.plotting as bkplt
import bokeh.models as bkmod
import plotly.express as px

from bokeh.io import output_notebook # enables plot interface in J notebook

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

In [ ]:
# Load cohorts to find sample query to select samples for this cohort.

df_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson").set_index("cohort_id")
cohort = df_cohorts.loc[cohort_id]
cohort

In [ ]:
# Initialise access to MalariaGEN data.

extra_params = dict()
if use_gcs_cache:
    extra_params["url"] = "simplecache::gs://vo_agam_release"
    extra_params["simplecache"] = dict(cache_storage=(here() / "gcs_cache").as_posix())

ag3 = malariagen_data.Ag3(
    # Pin the version of the cohorts analysis for reproducibility.
    cohorts_analysis=cohorts_analysis,
    results_cache=(here() / "malariagen_data_cache").as_posix(),
    **extra_params,
)
ag3

In [ ]:
# Load sample metadata for this cohort.

df_samples = ag3.sample_metadata(sample_query=cohort['sample_query'])
df_samples

In [ ]:
# Determine collection dates.

df_collection_dates = (
    df_samples
    .groupby(["year", "month"])
    .size()
    .reset_index()
    .rename(columns={0: 'count'})
)
df_collection_dates

In [ ]:
# Determine first and last collection months.

min_month = df_collection_dates["month"].min()
max_month = df_collection_dates["month"].max()

if min_month < 0:
    start_month = end_month = None
else:
    start_month = pd.to_datetime(min_month, format="%m").month_name()
    end_month = pd.to_datetime(max_month, format="%m").month_name()
    
start_month, end_month

In [ ]:
# Determine unique collection locations.

df_locations = df_samples[["location", "longitude", "latitude"]].drop_duplicates()
df_locations

In [ ]:
# Extract provenance information about the samples.

df_contributors = df_samples[["release", "sample_set", "contributor"]].drop_duplicates()
df_contributors["study"] = df_contributors.apply(
    lambda v: "Ag1000G" if v["sample_set"].startswith("AG1000G") else "TODO",
    axis="columns"
)
df_contributors["release"] = df_contributors["release"].apply(
    lambda v: f"Ag{v}"
)
df_contributors.rename(columns={
    "contributor": "Contributor",
    "study": "Study",
    "release": "Data release",
    "sample_set": "Sample set",
}, inplace=True)
df_contributors.set_index(["Contributor", "Study", "Data release"], inplace=True)
df_contributors

In [ ]:
# Build a paragraph with summary information about the samples in this cohort.

n_locations = len(df_locations)

summary_info = f"""This cohort comprises {cohort['cohort_size']:,} samples from the 
*{cohort['taxon']}* taxon, collected from {n_locations} locations within the administrative 
division of {cohort['admin2_name']}, {cohort['admin1_name']}, {cohort['country']}."""

if start_month and start_month == end_month:
    summary_info += f""" Collections were made in {start_month} {cohort['year']}."""
elif start_month:
    summary_info += f""" Collections were made between {start_month} and {end_month} in {cohort['year']}."""
else:
    summary_info += f""" Collections were made in {cohort['year']}."""
    
display(Markdown(summary_info))


## Selection scans

In [ ]:
# load signals to overlay on H12 plots

df_signals = [
    pd.read_csv(here() / "build/h12-signal-detection/" / f"{cohort_id}_{contig}.csv")
    for contig in contigs
]
df_signals = pd.concat(df_signals).assign(bottom=0, top=1)

In [ ]:
# load window sizes 
h12_calibration_dir = "build/h12-calibration"
with open(here() / h12_calibration_dir / f"{cohort_id}.yaml") as h12_calibration_file:
    h12_calibration_params = yaml.safe_load(h12_calibration_file)
h12_window_size = h12_calibration_params["h12_window_size"]
g123_calibration_dir = "build/g123-calibration"
with open(here() / g123_calibration_dir / f"{cohort_id}.yaml") as g123_calibration_file:
    g123_calibration_params = yaml.safe_load(g123_calibration_file)
g123_window_size = g123_calibration_params["g123_window_size"]

if cohort.taxon == 'arabiensis':
    phasing_analysis = 'arab'
else:
    phasing_analysis = 'gamb_colu'

ihs_window_size = 100

def plot_h12_g123_ihs_tracks(
        contig, 
        df_signals,
        sizing_mode='stretch_width', 
        show=False, 
        width=800, 
        genes_height=100
    ):

    sample_query = cohort["sample_query"]
    df = df_signals.query("contig == @contig")
    
    center_xs = [np.array([row.pcenter, row.pcenter]) for idx, row in df.iterrows()]
    center_ys = [np.array([row.level, row.level + .2]) for idx, row in df.iterrows()]

    source = bkmod.ColumnDataSource(data={
        'cohort': df.cohort_id,
        'chromosome': df.contig,
        'score': df.delta_i.astype(int),
        'peak_start': df.span2_pstart,
        'peak_stop': df.span2_pstop,
        'focus_start': df.focus_pstart,
        'focus_stop': df.focus_pstop,    
        'center_xs': center_xs,
        'center_ys': center_ys,
        'bottom': df.bottom ,
        'top': df.top,
        })
    
    fig1 = ag3.plot_h12_gwss_track(
        contig=contig, 
        window_size=h12_window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        show=show,
        width=width,
    )
    fig1.xaxis.visible = False
    
    quad = fig1.quad(bottom='bottom', top='top', left='peak_start', right='peak_stop', 
          source=source, color="gray", alpha=.1, line_width=1)
    quad2 = fig1.quad(bottom='bottom', top='top', left='focus_start', right='focus_stop', 
          source=source, color="red", alpha=.4)
    glyph = bkmod.MultiLine(xs='center_xs', ys='center_ys', line_color='red', line_width=2)
    fig1.add_glyph(source=source, glyph=glyph)
    
    hover = bkmod.HoverTool(tooltips=[
        ("Cohort", '@cohort'),
        ("Score", '@score'),
        ("Focus", "@focus_start{,} - @focus_stop{,}"),
    ], renderers=[quad])

    fig1.add_tools(hover)
    
    fig2 = ag3.plot_g123_gwss_track(
        contig=contig, 
        window_size=g123_window_size, 
        sites=phasing_analysis,
        site_mask=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig2.xaxis.visible = False

    fig3 = ag3.plot_ihs_gwss_track(
        contig=contig, 
        window_size=ihs_window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig3.xaxis.visible = False


    fig4 = ag3.plot_genes(
        region=contig, 
        show=show,
        sizing_mode=sizing_mode,
        width=width,
        height=genes_height,
        x_range=fig1.x_range
        )
                        
    fig = bklay.gridplot(
        [fig1, fig2, fig3, fig4],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode=sizing_mode,
    )
    return fig 

In [ ]:
for contig in contigs:
    
    display(HTML(f"<h3>Chromosome {contig}</h3>"))
    
    fig = plot_h12_g123_ihs_tracks(
        contig=contig,
        df_signals=df_signals,
    );

    bkplt.show(fig)


## Sampling information
### Collection locations

In [ ]:
df_samples[['latitude', 'longitude', 'taxon']].groupby(['latitude', 'longitude', 'taxon']).size().to_frame().rename(columns={0: 'count'}).reset_index()

In [ ]:
from ipyleaflet import Map, Marker, basemaps
from ipywidgets import HTML

center = cohort[['latitude', 'longitude']].to_list()
m = Map(center=center, zoom=9, basemap=basemaps.OpenTopoMap)

df = df_samples[['latitude', 'longitude', 'taxon']].groupby(['latitude', 'longitude', 'taxon']).size().to_frame().rename(columns={0: 'count'}).reset_index()

for coh_id, row in df.iterrows():
    lat, long = row[['latitude', 'longitude']]
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'

    marker = Marker(location=(lat, long), draggable=True, opacity=0.7, color=color)
    m.add_layer(marker);

    message2 = HTML()
    message2.value = f"n = {row['count']}"
    marker.popup = message2

display(m)

### Collection dates

In [ ]:
df_loc_dt = df_samples.pivot_table(
    index=["location", "longitude", "latitude"],
    columns="month",
    values="sample_id",
    aggfunc="count",
    fill_value=0,
)
# df_loc_dt.columns = [
#     pd.to_datetime(x, format="%m").month_name() for x in df_loc_dt.columns
# ]
df_loc_dt.reset_index(inplace=True)
cols = pd.MultiIndex.from_tuples(
    [("Location", "Name"), ("Location", "Longitude"), ("Location", "Latitude")] + 
    [("Month", pd.to_datetime(x, format="%m").month_name()) 
     for x in df_loc_dt.columns[3:]], 
#     names=["lvl0", "lvl1"]
)
df_loc_dt.columns = cols
# df_loc_dt.index.names = [x.capitalize() for x in df_loc_dt.index.names]
df_loc_dt_styled = (
    df_loc_dt
    .style
    .format(precision=3)
    .set_caption("No. samples by location and month of collection.")
    .hide()
)
df_loc_dt_styled



In [ ]:
# if month_collections:
#     fig = px.bar(df_collection_dates, x='month', y='count', title=f"Collection dates {quarter} {year}")
#     display(fig)
# else:
#     display(Markdown("No per-month collection data exists for this cohort."))

## Diagnostics
### H12 calibration

In [ ]:
display(Markdown(f"Selected window size: **{h12_window_size:,}**"))

window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)

ag3.plot_h12_calibration(
    contig=h12_calibration_contig,
    analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort['sample_query'],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);


### G123 Calibration

In [ ]:
display(Markdown(f"Selected window size: **{g123_window_size:,}**"))

ag3.plot_g123_calibration(
    contig=h12_calibration_contig,
    sites=phasing_analysis,
    site_mask=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort['sample_query'],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

## Data sources

In [ ]:
df_sources_style = (
    df_contributors
    .style
    .set_caption("MalariaGEN Vector Observatory partners, studies and sample sets contributing data for this cohort.")
)
df_sources_style